## 1. Pyspark 현재 사양

In [2]:
import findspark

findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="SparkContext")

# SparkContext 버전
print("스파크 컨텍스트 버젼: ", sc.version)

# SparkContext 파이썬 버전
print("Spark Context 파이썬 버전:", sc.pythonVer)

# SparkContext 마스터
print("Spark Context 마스터:", sc.master)


스파크 컨텍스트 버젼:  3.3.4
Spark Context 파이썬 버전: 3.9
Spark Context 마스터: local[*]


## 2. pyspark 환경 설정 및 간단한 예제

In [9]:
import findspark

findspark.init()
# pyspark 환경 빌드 예제
from pyspark.sql import SparkSession

#sparksession 드라이버 프로세스 얻기
#클러스터모드의 경우 master에 local[*] 대신 yarn이 들어간다.
spark = SparkSession.builder.appName("sample").master("local[*]").getOrCreate()

#jupyter환경에서만 가능한 config, .show()메소드를 사용할 필요없이 dataframe만 실행해도,정렬된 프린팅을 해준다.
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)


In [3]:
#간단한 예제 1
df= spark.range(500).toDF("number")
df.select(df["number"]+10)

In [4]:
#간단한 예제 2
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

In [10]:
spark.stop()

## 3. json 파일 불러오기 실습

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
sp = SparkSession.builder.appName("file_load").master("local[*]").getOrCreate()

In [2]:
# csv_file_path = r"C:\Users\psych\turnover.csv"
# # CSV 파일 읽기
# df = sp.read.csv(csv_file_path, header=True, inferSchema=True)

# # 데이터프레임 출력
# df.show()

In [59]:
json_file_path = r"C:\Users\psych\GRANDMASTER_01170808.json"
df1 = sp.read.json(json_file_path)
# df1.printSchema()

In [27]:
df1.show()

In [58]:
from pyspark.sql.functions import col,explode
participants_df = df1.select(col("mat_info.metadata.participants"))
# participants_df.show()


# explode를 사용하면 배열 안의 각 요소가 별도의 행으로 분리되어 표시
exploded_participants_df = participants_df.select(explode("participants").alias("participant"))
# exploded_participants_df.show(truncate=False)

### spark DF에서 데이터 추출하기

row 단위로만 가능함

In [80]:
from pyspark.sql.functions import monotonically_increasing_id


selected_columns = ["mat_info.info.participants.summonerName","mat_info.info.participants.assists","mat_info.info.participants.teamPosition" ]
new_df = df1.select(*selected_columns)
summon_df = new_df.select(explode("summonerName").alias("summonerName"))
summon_df = summon_df.withColumn("row_id", monotonically_increasing_id())
# new_df.printSchema()
# new_df.show()

In [81]:
team_df = new_df.select(explode("teamposition").alias("teamposition"))
team_df = team_df.withColumn("row_id", monotonically_increasing_id())

In [83]:
result_df = summon_df.join(team_df, "row_id", "inner").drop("row_id")

In [84]:
result_df.show()

+----------------+------------+
|    summonerName|teamposition|
+----------------+------------+
|아름다워지고싶요|         TOP|
|      warangurus|      JUNGLE|
|        DK Saint|      MIDDLE|
|        DK Rahel|      BOTTOM|
|         용병456|     UTILITY|
|    General irel|         TOP|
|             Zac|      JUNGLE|
|          쏘령관|      MIDDLE|
|          바이탈|      BOTTOM|
|                |     UTILITY|
|챔프처음해도잘함|         TOP|
|나는 멍청이 우우|      JUNGLE|
|      쿠쿠공주님|      MIDDLE|
|         ADC0315|      BOTTOM|
|            neuo|     UTILITY|
|    No teammates|         TOP|
|    gnoeiwngonge|      JUNGLE|
|       HLE Loki1|      MIDDLE|
|                |      BOTTOM|
|      비밀인간jk|     UTILITY|
+----------------+------------+
only showing top 20 rows



In [85]:
# 스파크 세션 중단
sp.stop()